1.苯分子的CASCI练习

In [14]:
#1.读取文件中的相关积分的数值并保存
import numpy as np
import math
import itertools
h1=np.load("D:\\python\\scf\\1e.npy..bin")
h2=np.load("D:\\python\\scf\\2e(1).npy..bin")
np.shape(h1)
np.shape(h2)



(36, 36, 36, 36)

In [15]:
#2.将原子轨道积分表来表示分子轨道的积分
cx=np.load("D:\\python\\scf\\xishu3.npy")
#print(xishu)尝试用函数形式来修改单双电子算符的积分
h1_transformed = np.dot(cx.T, h1)
h1m = np.dot(h1_transformed, cx)
#双电子积分项的转换，这里必须得使用einsum函数来简化计算过程，优化代码。
def double(cx,h2):
    T = np.einsum('mp,mnlk->pnlk', cx, h2)  # 结果维度 (P, N, L, K)
    A = np.einsum('nq,pnlk->pqlk', cx, T)
    B = np.einsum('lr,pqlk->pqrk', cx, A)
    D = np.einsum('ks,pqrk->pqrs', cx, B)
    return D
h2m=double(cx,h2)
#测试
np.shape(h1m)
np.shape(h2m)
np.save('h2m',h2m)

In [16]:
#测试
def E0(h1m,h2m):
    E0=0.0
    for i in range(21):
        E0+=h1m[i,i]
    E0=E0*2
    for i in range(21):
        for j in range(21):
            E0+=2*h2m[i,i,j,j]-h2m[i,j,j,i]
    return E0
E0=E0(h1m,h2m)
E1 =203.030749104     
Etot = E0+E1 
Etot   

-227.88940992315554

In [17]:
#3.冻结核能量计算
#这里的错误是对活性空间的索引序号。活性空间的轨道索引应该是[17,20-24]但在python上应该写成16,19,20,21,22,23
active_orbitals = [16,19,20,21,22,23]
frozen_orbitals = list(range(16)) + [17, 18]#注意对冻结空间，与活性空间的标号
h1_active = h1m[np.ix_(active_orbitals, active_orbitals)]#初始化活性空间的单电子积分
h2_active = h2m[np.ix_(active_orbitals,active_orbitals,active_orbitals,active_orbitals)]
#冻结核能量计算
def E_fc(h1m,h2m):
    frozen_orbitals = list(range(16)) + [17, 18]#冻结核的轨道索引
    E0=0.0
    for i in frozen_orbitals:
        E0+=2*h1m[i,i]
        for j in frozen_orbitals:
           E0+=2*h2m[i,i,j,j]-h2m[i,j,j,i]
    return E0
E_fc=E_fc(h1m,h2m)
print(E_fc)
np.save('h2_active',h2_active)

#density=calc_P_rhf(cx)


-424.57859283365116


In [18]:
#4.活性空间中对单电子积分项的修改，以及双电子积分项得截取
#h_active = h_AO[np.ix_(active_orbitals, active_orbitals)]对应着np.ix函数
# 活性空间单电子积分修正
frozen_orbitals = list(range(16)) + [17, 18]#注意对冻结空间，与活性空间的标号
def single_correction(h1_active, frozen_orbitals, active_orbitals, h2m):
    n_active = len(active_orbitals)
    h1_active_correction = np.zeros((n_active, n_active))  # 修正矩阵
    for p, i in enumerate(active_orbitals):  # i是全局轨道，p是局部索引
        for q, j in enumerate(active_orbitals):  # j是全局轨道，q是局部索引
            correction = 0.0
            for m in frozen_orbitals:  # 遍历冻结轨道
                correction += 2 * h2m[i, j, m, m] - h2m[i, m, m, j]
            h1_active_correction[p, q] = correction  # 存储修正值
    return h1_active + h1_active_correction  # 返回修正后的单电子积分
h1_active_cor = single_correction(h1_active,frozen_orbitals,active_orbitals,h2m)
print("修正后的活性空间单电子积分矩阵:")
h1_active_cor
np.save('h1-_active_cor',h1_active_cor)

修正后的活性空间单电子积分矩阵:


In [19]:
#5. Compare the excitation, i.e. the mismatching orbital indices
def compare_excitation(left_indices, right_indices):
    left_indices_set = set(left_indices)
    right_indices_set = set(right_indices)

    # elements in the left set that are not elements in the right one
    unique_from_left = left_indices_set - right_indices_set#注意集合元素的差集的定义，得到的是左边有而右边没有的元素
    unique_from_right = right_indices_set - left_indices_set
    return (unique_from_left, unique_from_right)   

In [20]:
a=compare_excitation('01234','12356')
print(a)

({'0', '4'}, {'5', '6'})


In [21]:
b=compare_excitation('012','215')
print(b)
                     

({'0'}, {'5'})


In [22]:
#6.相位因子的获取
def phase_factor(excitation, left_indices, right_indices):
    indices_swap = 0
    left_excitation, right_excitation = excitation

    # check the left and right indices are in ascending order
    assert(sorted(left_indices) and sorted(right_indices))

    # check the excitations are in ascending order
    assert((sorted(left_excitation) or len(left_excitation) == 0)
           and (sorted(right_excitation) or len(right_excitation) == 0))

    # For a list of numbers in ascending order (which is checked in the assertions above), the number of swaps needed
    # to move a number to the head of the list is determined by its index in the list. e.g. in the following list
    # 1 4 5 6 8
    #       ^
    # 0 1 2 3 4  <---- index of the list in python
    # you need three 2-cycles to move 6 to the head of the list.
    # If you have two or more numbers, you need to remove the redundant swapping to keep the operators
    # corresponding to excited orbitals in ascending order. e.g.
    # 1 4 5 6 8
    #       ^ ^
    # 0 1 2 3 4  <---- index of the list in python
    # to move the two numbers we first add the indices of two numbers in the list to get the number of swaps for the
    # list
    # 8 6 1 4 5
    # and one step is redundant because we had 8 before 6, to correct this we subtract 1 from the number of swaps to
    # represent
    # 6 8 1 4 5
    # and this subtraction corresponds to the index of the excitation orbital in excitation list.
    for index, orbital_index in enumerate(left_excitation):
        indices_swap += left_indices.index(orbital_index) - index

    for index, orbital_index in enumerate(right_excitation):
        indices_swap += right_indices.index(orbital_index) - index

    return math.pow(-1, indices_swap)


In [23]:
#7.slaterg规则的编写，以及FCI矩阵的获取
def ci_hamiltonian_in_sparse_matrix(one_electron_integrals, two_electron_integrals, n_elecs, n_spin = 0) :

    n_rows, n_cols = one_electron_integrals.shape

    n_orbs = n_rows

    assert(n_rows == n_cols)
    assert(np.all(np.array(two_electron_integrals.shape) == n_orbs))

    n_alpha = (n_elecs + n_spin) // 2
    n_beta = (n_elecs - n_spin) // 2

    alpha_combinations = [list(x) for x in itertools.combinations(range(n_orbs), n_alpha)]
    beta_combinations = [list(x) for x in itertools.combinations(range(n_orbs), n_beta)]

    n_dim = len(alpha_combinations) * len(beta_combinations)

    H_active = np.zeros((n_dim,n_dim))

    for i in range(n_dim):
        i_alpha_combination = alpha_combinations[i // len(beta_combinations)]#对α的索引是用i整除β串的余数
        i_beta_combination = beta_combinations[i %  len(beta_combinations)]#对β的索引是地板除数

        for j in range(i, n_dim):#保证避免重复j<i

            j_alpha_combination = alpha_combinations[j // len(beta_combinations)]
            j_beta_combination = beta_combinations[j % len(beta_combinations)]

            alpha_excitation = compare_excitation(i_alpha_combination, j_alpha_combination)#左右两边α和β串的分开比较
            beta_excitation = compare_excitation(i_beta_combination, j_beta_combination)

            n_alpha_excitation = len(alpha_excitation[0])
            n_beta_excitation = len(beta_excitation[0])

            # more than two electrons are excited - the matrix element is zero
            if n_alpha_excitation + n_beta_excitation > 2:
                continue

            # the phase factor for alpha and beta each
            alpha_phase = phase_factor(alpha_excitation, i_alpha_combination, j_alpha_combination)
            beta_phase = phase_factor(beta_excitation, i_beta_combination, j_beta_combination)

            # I think of the alignment of the creation operators as something like
            #  | beta |  | alpha |
            #   1  3  5   2  5  6  |0>
            total_phase_factor = alpha_phase * beta_phase # 确保此值是及时更新的

            # No excitation, i.e. the Slater determinant is the same on the two sides
            if n_alpha_excitation + n_beta_excitation == 0:#对应着情形一

                one_electron_part = \
                    np.einsum("ii->", one_electron_integrals[np.ix_(i_alpha_combination, i_alpha_combination)]) \
                    + np.einsum("ii->", one_electron_integrals[np.ix_(i_beta_combination, i_beta_combination)])
                    
                # <ij | v | ij>, or (ii | jj). Non trivial contribution from configurations having
                # the same spin for i and the same spin for j
                
                coulomb_part = \
                    np.einsum("iijj->", two_electron_integrals[np.ix_(i_alpha_combination, i_alpha_combination,
                                                                      i_beta_combination, i_beta_combination)]) \
                    + 0.5 * np.einsum("iijj->", two_electron_integrals[np.ix_(i_alpha_combination, i_alpha_combination,
                                                                              i_alpha_combination, i_alpha_combination)]) \
                    + 0.5 * np.einsum("iijj->", two_electron_integrals[np.ix_(i_beta_combination, i_beta_combination,
                                                                              i_beta_combination, i_beta_combination)])
                # <ij | v | ji>, or (ij | ji).
                # i and j must have the same spin, and thus the mixed spin terms are omitted
                exchange_part = \
                    0.5 * np.einsum("ijji->", two_electron_integrals[np.ix_(i_alpha_combination, i_alpha_combination,
                                                                            i_alpha_combination, i_alpha_combination)]) + \
                    0.5 * np.einsum("ijji->", two_electron_integrals[np.ix_(i_beta_combination, i_beta_combination,
                                                                            i_beta_combination, i_beta_combination)])

                element = one_electron_part + coulomb_part - exchange_part
                H_active[i,j] = element*total_phase_factor

            if n_alpha_excitation + n_beta_excitation == 1:
                #集合中的.intersection用于返回两个或多个集合的交集
                alpha_shared_orbitals = list(set(i_alpha_combination).intersection(set(j_alpha_combination)))
                beta_shared_orbitals = list(set(i_beta_combination).intersection(set(j_beta_combination)))

                concatenated = alpha_shared_orbitals + beta_shared_orbitals#将α电子和β电子的共享轨道拼接起来然后
                                                                            #得到一个共同的轨道集合，后续在计算双电子积分时会用到

                if n_alpha_excitation == 1:#处理α电子激发的情况

                    index_a = list(alpha_excitation[0])[0]
                    index_b = list(alpha_excitation[1])[0]

                    # <a i | v | b i>, or (ab | ii).
                    coulomb_submatrix = two_electron_integrals[np.ix_([index_a], [index_b], concatenated, concatenated)]

                    # <a i | v | i b>, or (ai | ib).
                    # a and i must have same spin (a and b are already have the same spin)
                    exchange_submatrix = two_electron_integrals[
                        np.ix_([index_a], alpha_shared_orbitals, alpha_shared_orbitals, [index_b])]

                else:#处理β电子激发的情况
                    index_a = list(beta_excitation[0])[0]
                    index_b = list(beta_excitation[1])[0]

                    coulomb_submatrix = two_electron_integrals[np.ix_([index_a], [index_b], concatenated, concatenated)]
                    exchange_submatrix = two_electron_integrals[
                        np.ix_([index_a], beta_shared_orbitals, beta_shared_orbitals, [index_b])]

                element = \
                    one_electron_integrals[index_a, index_b] + \
                          np.einsum("ijkk->", coulomb_submatrix) - np.einsum("ikkj->", exchange_submatrix)

                H_active[i,j] = total_phase_factor * element

            if n_alpha_excitation == 2:#双重激发分成了三种情况，两个α电子激发的情况，两个β电子激发的情况，一个α一个β激发的情况
                left_excitation, right_excitation = map(list, alpha_excitation)#对 beta_excitation 的每个元素调用 list 函数，将其转换为列表。例如，如果 beta_excitation = ((2, 3), (4, 5))，执行 map(list, beta_excitation) 的结果是 [ [2, 3], [4, 5] ]。


                # <ab | v | xy> - <ab | v | yx>, or (ax | by) - (ay | bx)
                element = two_electron_integrals[left_excitation[0], right_excitation[0],
                                                 left_excitation[1], right_excitation[1]] \
                          - two_electron_integrals[left_excitation[0], right_excitation[1],
                                                   left_excitation[1], right_excitation[0]]

                H_active[i,j] = total_phase_factor * element

            if n_beta_excitation == 2:
                left_excitation, right_excitation = map(list, beta_excitation)

                # <ab | v | xy> - <ab | v | yx>, or (ax | by) - (ay | bx)
                element = two_electron_integrals[left_excitation[0], right_excitation[0],
                                                 left_excitation[1], right_excitation[1]] \
                          - two_electron_integrals[left_excitation[0], right_excitation[1],
                                                   left_excitation[1], right_excitation[0]]

                H_active[i,j] = total_phase_factor * element

            if n_alpha_excitation == 1 and n_beta_excitation == 1:
                a = list(alpha_excitation[0])[0]
                b = list(beta_excitation[0])[0]
                x = list(alpha_excitation[1])[0]
                y = list(beta_excitation[1])[0]

                element = two_electron_integrals[a, x, b, y]

                H_active[i,j] = total_phase_factor * element


    return  H_active


In [24]:
#8.迭代对角化得到相应的不同能级的能量
# 问题一：计算活性空间(6o,6e)（17,20-24号轨道, 3alphae, 3beta e) 时的state0，state1 的能量。
from scipy.sparse.linalg import eigsh
E1 =203.030749104 
H_active=ci_hamiltonian_in_sparse_matrix(h1_active_cor,h2_active,6,n_spin=0)

assert np.allclose(H_active, np.triu(H_active))
#H = H_active + H_active.T - H_active.diagonal()
H = H_active + H_active.T
np.fill_diagonal(H , np.diag(H_active))#由上三角矩阵得到全对称矩阵
eigenvalues, eigenvectors = eigsh(H, k=5, which='SA') 
sorted(eigenvalues)
state0=E1+E_fc+eigenvalues[0]
print('E(state0) =',state0)
#可见整个的流程已经完成了，明显的bug已经除掉。现在的任务是怎样解决隐藏的bug
state1=E1+E_fc+eigenvalues[1]
print('E(state1) =',state1)

H[:10,:10]

E(state0) = -227.9952440578978
E(state1) = -227.84865300100614


array([[-6.34156619e+00, -4.85724242e-06,  1.24287696e-06,
         2.11172746e-08,  7.94679851e-06,  2.87326987e-06,
        -1.17936325e-07, -6.29660490e-02,  2.18205661e-06,
         1.08546884e-05],
       [-4.85724242e-06, -6.04813205e+00,  3.54066555e-04,
        -3.69127203e-05, -3.54182159e-04, -3.73171142e-02,
        -1.65251291e-05, -8.92673040e-08,  3.30962414e-02,
         1.74076997e-02],
       [ 1.24287696e-06,  3.54066555e-04, -6.03638363e+00,
        -1.41942524e-05,  2.56489348e-02,  3.54961636e-04,
         3.93351097e-05, -1.41930795e-04,  1.74116518e-02,
        -3.30997474e-02],
       [ 2.11172746e-08, -3.69127203e-05, -1.41942524e-05,
        -5.82882541e+00, -1.87071857e-05,  2.84804213e-05,
         3.35200367e-07,  3.95202787e-06, -4.71293144e-05,
         5.10173911e-06],
       [ 7.94679851e-06, -3.54182159e-04,  2.56489348e-02,
        -1.87071857e-05, -6.03640208e+00, -3.53468401e-04,
         1.50218412e-05,  1.27903012e-04,  1.74094456e-02,
        -3.

In [25]:
#9.迭代对角化得到相应的不同能级的能量
# 问题二：计算活性空间(6o,6e)（17,20-24号轨道, 4alphae, 2beta e) 时的state0，state1 的能量。
from scipy.sparse.linalg import eigsh
E1 =203.030749104 
H_active=ci_hamiltonian_in_sparse_matrix(h1_active_cor,h2_active,6,n_spin=2)
H = H_active + H_active.T
np.fill_diagonal(H , np.diag(H_active))#由上三角矩阵得到全对称矩阵
eigenvalues, eigenvectors = eigsh(H, k=5, which='SA') 
sorted(eigenvalues)
state0=E1+E_fc+eigenvalues[0]
print('E(state0) =',state0)


E(state0) = -227.8486530010061


In [26]:
# 4.利用Slater-Condon rules，通过对不同的slater态进行对比来对分子积分表进行索引。

#def compute_hamiltonian_element(det1, det2, h1m, h2m):

    #alpha1, beta1 = det1
    #alpha2, beta2 = det2
    
    #if det1 == det2:  # Diagonal elements
        # 1-electron contribution
        #h1 = sum(h1e[p, q] for p in alpha1 for q in alpha1) + \
             #sum(h1e[p, q] for p in beta1 for q in beta1)
        
        # 2-electron contribution
        #h2 = sum(h2e[p, q, r, s] for p in alpha1 for q in alpha1 for r in alpha1 for s in alpha1) + \
             #sum(h2e[p, q, r, s] for p in beta1 for q in beta1 for r in beta1 for s in beta1) + \
             #sum(h2e[p, q, r, s] for p in alpha1 for q in beta1 for r in alpha1 for s in beta1)
        
        #return h1 + 0.5 * h2
    
    # Off-diagonal elements
    #return 0  

# Construct the Hamiltonian matrix
#H = np.zeros((12,12))
#for i, det1 in enumerate(slater_dets):
    #for j, det2 in enumerate(slater_dets):
        #H[i, j] = compute_hamiltonian_element(det1, det2, h1e, h2e)

# Diagonalize the Hamiltonian
#eigenvalues, eigenvectors = np.linalg.eigh(H)

# Extract the ground-state energy
#ground_state_energy = eigenvalues[0]
#ground_state_energy
